In [2]:
import pandas as pd
import numpy as np
import s3fs
import boto3

In [207]:
s3 = boto3.client('s3')
csw = pd.read_csv('s3://isfl-surrender-bot/procSW.csv')
curS = csw.S
curW = csw.W

career = pd.read_csv('s3://isfl-surrender-bot/AllStats/career.csv')
careerRanks = pd.read_csv('s3://isfl-surrender-bot/AllStats/Ranks/career.csv')
season = pd.read_csv('s3://isfl-surrender-bot/AllStats/season.csv').set_index('S')
game = pd.read_csv('s3://isfl-surrender-bot/AllStats/game.csv')

/var/folders/8f/0znsb09d3cz3qvv5tv_3n53c0000gn/T/ipykernel_95336/3816781461.py:9: DtypeWarning: Columns (64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  game = pd.read_csv('s3://isfl-surrender-bot/AllStats/game.csv')


In [208]:
passStats = ['Pass Yds', 'Pass Avg', 'Pass TD', 'Pass Int', 'Pass Rat', 'Pass Cmp', 'Pass Att', 'Pass Pct']
rushStats = ['Rush Att', 'Rush Yds', 'Rush Avg', 'Rush TD']
recStats = ['Rec Rec', 'Rec Yds', 'Rec Avg', 'Rec TD']
kickStats = ['K XPM', 'K XPA', 'K XPPct', 'K FGM', 'K FGA', 'K FGPct', 'K FG_U20 Pct', 'K FG_2029 Pct', 'K FG_3039 Pct', 'K FG_4049 Pct', 'K FG_50 Pct']
puntStats = ['P Punts', 'P Yds', 'P Avg', 'P Lng', 'P Inside 20','P In20Rate',]
defStats = ['DEF Tck', 'DEF TFL', 'DEF Sack', 'DEF PD', 'DEF Int', 'DEF Sfty', 'DEF TD', 'DEF FF', 'DEF FR', 'DEF Blk P', 'DEF Blk XP', 'DEF Blk FG']
allStats = ['AllPurpose Yds', 'AllPurpose TD', 'Scrimmage Yds', 'Scrimmage TD', 'Points']
stStats = ['ST KR', 'ST KRYds', 'ST PRYds', 'ST KRAvg', 'ST PRAvg', 'ST KR_TD', 'ST PR_TD', 'ST PR', 'ST KRLng', 'ST PRLng']
olStats = ['Other Pancakes', 'Other Sacks Allowed']
otherStats = ['Other Penalties', 'Other Yards']

In [209]:
sampleID = career.loc[career.FullName == 'Busch Goose','PlayerID'].iloc[0]
playerCareer = career.loc[career.PlayerID == sampleID].copy()
playerCareer['Team'] = 'Career'
playerRanks = careerRanks.loc[careerRanks.PlayerID == sampleID].copy().fillna('DNQ').apply(pd.to_numeric,errors='ignore',downcast='integer').astype(str)
playerRanks['Team'] = 'Rank'
playerTeams = game.loc[game.PlayerID == sampleID].groupby(['S','Team']).sum(numeric_only=True)
playerSeasons = season.loc[season.PlayerID == sampleID].copy()
teamSeason = pd.DataFrame(['/'.join(list(playerTeams.reset_index().set_index('S').loc[l,'Team'])) if playerTeams.reset_index().set_index('S').index.value_counts().loc[l] > 1 else playerTeams.reset_index().set_index('S').loc[l,'Team'] for l in list(playerSeasons.index)],columns=['Team'],index=playerSeasons.index)
playerDF = pd.concat([pd.concat([teamSeason,playerSeasons],axis=1).reset_index(names=['S']),playerRanks,playerCareer]).fillna('')

In [210]:
playerDF

,S,Team,PlayerID,FullName,Pass Yds,Pass Avg,Pass TD,Pass Int,Pass Rat,Pass Cmp,...,K FG_2029 Pct,K FG_3039 Pct,K FG_4049 Pct,K FG_50 Pct,AllPurpose Yds,Scrimmage Yds,AllPurpose TD,Scrimmage TD,Points,Rec Tar.1
0,28.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1074.0,1074.0,11.0,11.0,66.0,
1,29.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1603.0,931.0,11.0,11.0,66.0,
2,30.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2236.0,1237.0,6.0,6.0,36.0,
3,31.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2392.0,1484.0,11.0,11.0,66.0,
4,32.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1836.0,1123.0,11.0,11.0,66.0,
5,33.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1907.0,1166.0,10.0,10.0,60.0,
6,34.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1940.0,1161.0,6.0,6.0,36.0,
7,35.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2637.0,1776.0,15.0,15.0,90.0,
8,36.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2716.0,1774.0,14.0,13.0,84.0,
9,37.0,BAL,16906.0,Busch Goose,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1943.0,1522.0,11.0,11.0,66.0,


In [198]:
season.loc[season.FullName == 'Busch Goose','ST KRLng']

S
28    101.0
29    101.0
30    101.0
31    101.0
32    101.0
33    101.0
34    101.0
35    101.0
36    101.0
37    101.0
38    101.0
39    101.0
Name: ST KRLng, dtype: float64

In [162]:
if all([((playerDF[o] == 0).all()) | (playerDF[o].isnull().all()) for o in otherStats]):
    print('none')
else:
    print('yay')

yay
